# Задание №2 "Spotify"

## Импорт библиотек

In [ ]:
import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.neighbors import KNeighborsRegressor

from sklearn.inspection import permutation_importance

import tensorflow as tf

from tensorflow import keras

## Загрузка данных

In [ ]:
df = pd.read_csv('/kaggle/input/spotify/dataset.csv')

In [ ]:
df.info()

**Описание столбцов**

<ul><li><strong>artists</strong>: Имя или имена исполнителя(ей) трека.</li><li><strong>album_name</strong>: Название альбома, в котором присутствует трек.</li><li><strong>track_name</strong>: Название трека или название песни.</li><li><strong>popularity</strong>: Оценка популярности трека, обычно основанная на метриках прослушиваний.</li><li><strong>duration_ms</strong>: Продолжительность трека в миллисекундах.</li><li><strong>explicit</strong>: Указатель на наличие откровенного (нецензурного) контента; обычно представляется как "True" или "False".</li><li><strong>danceability</strong>: Показатель от 0.0 до 1.0, описывающий, насколько трек подходит для танцев, основываясь на темпе, стабильности ритма, силе удара и т.д.</li><li><strong>energy</strong>: Показатель от 0.0 до 1.0, отражающий интенсивность и активность трека.</li><li><strong>key</strong>: Тональность трека, представленная в виде целого числа, соответствующего музыкальным нотам (0=До, 1=До#/Реb и т.д.).</li><li><strong>loudness</strong>: Общая громкость трека в децибелах (дБ), где более высокие значения означают более громкие треки.</li><li><strong>mode</strong>: Модальность трека, где 1 указывает на мажор, а 0 — на минор.</li><li><strong>speechiness</strong>: Показатель от 0.0 до 1.0, показывающий наличие разговорных фрагментов, где более высокие значения обозначают более речеподобные треки.</li><li><strong>acousticness</strong>: Показатель от 0.0 до 1.0, предсказывающий, является ли трек акустическим.</li><li><strong>instrumentalness</strong>: Показатель от 0.0 до 1.0, оценивающий вероятность отсутствия вокала в треке.</li><li><strong>liveness</strong>: Показатель от 0.0 до 1.0, указывающий на присутствие аудитории; более высокие значения предполагают запись с концерта.</li><li><strong>valence</strong>: Показатель от 0.0 до 1.0, описывающий музыкальную позитивность трека (веселые, жизнерадостные звуки имеют более высокие значения).</li><li><strong>tempo</strong>: Общий темп трека в ударах в минуту (BPM).</li><li><strong>time_signature</strong>: Предполагаемая размерность трека (например, 4 соответствует размеру 4/4).</li><li><strong>track_genre</strong>: Жанр или стилистическая категория, связанная с треком.</li></ul>

In [ ]:
df

## Очистка данных

### Удаление лишних столбцов

In [ ]:
df.info()

In [ ]:
# Удаление стобцов 'Unnamed: 0' и "track_id", потому что они не несут никакой важной информации

df = df.drop(columns=["Unnamed: 0", "track_id", "track_name"])

In [ ]:
df.info()

### Очистка пустых значений

#### 1. Проверяем датасет на наличие пустых значений




In [ ]:
df.isnull().sum()

Как мы видим, в датасете очень мало пустых значений, так что строчки с пустыми значениями можно просто удалить.

#### 2. Удаление строк с пустыми значениями

In [ ]:
df = df.dropna()

In [ ]:
df.isnull().sum()

Как мы видим, больше пустых значений в датасете нет.

## Exploratory Data Analysis (EDA)

### Распределение популярности песен

In [ ]:
plt.figure(figsize=(10, 6))

sns.histplot(df['popularity'], bins=100, kde=True)

plt.title('Распределение популярности песен')

plt.xlabel('Популярность')

plt.ylabel('Частота')

plt.show()




**Вывод:** 

Большинство музыкальных треков остается незамеченными с нулевой популярностью.

Можно выделить категории популярности:  
* unpopular ( popularity = 0 )

* low_popularity ( popularity = (0, 20] )

* mid_popularity ( popularity = (20, 60] )

* high_popularity ( popularity = (60, 100] )








### Исследование метрики "продолжительность (duration_ms)"

#### 1. Создадим дополнительный столбец - продолжительность в секундах, чтобы было легче анализировать продолжительность.

In [ ]:
df['duration_s'] = df['duration_ms'] / 1000

#### 2. Проверим распределение продолжительности треков в датасете

In [ ]:
df['duration_s'].describe()

In [ ]:
plt.figure(figsize=(10, 6))

sns.histplot(df['duration_s'], bins=100, kde=True)

plt.title('Распределение длительности песен')

plt.xlabel('Длительность')

plt.ylabel('Частота')

plt.show()

In [ ]:
df[df['duration_s'] > 1000].shape[0]

In [ ]:
df[df['duration_s'] > 500].shape[0]

In [ ]:
df[df['duration_s'] <= 1000].shape[0]

In [ ]:
df[df['duration_s'] <= 500].shape[0]

**Вывод:**

Судя по полученным данным анализа и графику, мы видим, что в датасете присутствуют выбросы по длительности. Это песни, у которых длительность превышает 500 секунд, то есть примерно 8 минут и их популярность около нулевая.

Можно пренебречь треками длина которых превышает 500 секунд.


#### 3. Проверим как длительность влияет на популярность трека

In [ ]:
# Создаем таблицу с треками длительность которых меньше 500 секунд

df_filtered = df[df['duration_s'] <= 500]

In [ ]:
plt.figure(figsize=(10, 6))

sns.kdeplot(data=df_filtered, x="duration_s", y="popularity", cmap="viridis", fill=True)

max_duration = df_filtered['duration_s'].max()

plt.xticks(range(0, int(max_duration) + 50, 50))

plt.title("Плотность зависимости популярности от продолжительности трека")

plt.xlabel("Продолжительность (с)")

plt.ylabel("Популярность")

plt.show()

**Вывод:**

Авторы стараются создавать песни, которые смогут попасть на радио и стать популярными, поэтому делают их длительностью примерно 3 минуты.

Судя по графику существует 2 большие группы треков по продолжительности:

1. Низкая популярность и продолжительность от ~ 150 до 250 секунд (2.5 - 4 минут).

2. Средняя популярность ~ 40 - 50 и продолжительность ~ 175 до 225 секунд ("Радиоформат" - примерно 3 минуты).


Можно выделить категории продолжительности:

* short - [0; 120] секунд

* standart - (120; 240] секунд

* long - (240; ...] секунд


### Зависимость популярности от наличия откровенного контента

In [ ]:
df["explicit"].describe()

In [ ]:
plt.figure(figsize=(8, 6))

sns.boxplot(data=df, x="explicit", y="popularity")

plt.title("Зависимость популярности от наличия откровенного контента")

plt.xlabel("Наличие откровенного контента")

plt.ylabel("Средняя популярность")

plt.xticks([0, 1], ["Нет", "Да"])

plt.show()

**Вывод:**

Популярность треков с нецензурной лексикой выше, это может свидетельствовать о предпочтении слушателей к более нецензурному контенту. Однако, разница минимальна, а значит, что наличие нецензурной лексики не является ключевым фактором популярности.




### Зависимость популярности от танцевальности

In [ ]:
df["danceability"].describe()

In [ ]:
plt.figure(figsize=(10, 6))

sns.kdeplot(data=df, x="danceability", y="popularity", cmap="viridis", fill=True)

plt.title("Плотность зависимости популярности от танцевальности трека")

plt.xlabel("Танцевальность")

plt.ylabel("Популярность")

plt.show()

In [ ]:
# Создаем интервалы для танцевальности

bins = np.linspace(0, 1, 21)

df['danceability_bins'] = pd.cut(df['danceability'], bins=bins, include_lowest=True)



# Средняя популярность по каждому интервалу танцевальности

danceability_popularity = df.groupby('danceability_bins')['popularity'].mean()



plt.figure(figsize=(12, 6))

sns.lineplot(x=danceability_popularity.index.astype(str), y=danceability_popularity.values, marker="o", color="purple")

plt.title("Средняя популярность по интервалам танцевальности")

plt.xlabel("Интервалы танцевальности")

plt.ylabel("Средняя популярность")

plt.xticks(rotation=45)

plt.show()


**Вывод:**

* Большинство треков находится в диапазоне от 0.5 до 0.8 по танцевальности.

* Средняя популярность растет с увеличением танцевальности до 0.45–0.55, где достигает пика.
  
* После танцевальности 0.55–0.6 средняя популярность начинает плавно снижаться.

Можно выделить категории:



* Сбалансированные треки (0.4–0.6): Наиболее популярные.

* Сложные для восприятия (0.0–0.3): Могут быть использованы в узких жанрах или для специфических аудиторий.

* Ограниченно популярные (0.7–1.0): Подходят для танцевальных вечеринок, но ограничены в массовом успехе.



### Зависимость популярности от энергичсности

In [ ]:
df["energy"].describe()

In [ ]:
plt.figure(figsize=(10, 6))

sns.kdeplot(data=df, x="energy", y="popularity", cmap="viridis", fill=True)

plt.title("Плотность зависимости популярности от уровня энергичности трека")

plt.xlabel("Энергичность")

plt.ylabel("Популярность")

plt.show()

In [ ]:
# Создаем интервалы для уровня энергичности
bins = np.linspace(0, 1, 21)

df['energy_bins'] = pd.cut(df['energy'], bins=bins, include_lowest=True)



# Средняя популярность по каждому интервалу уровня энергичности

energy_popularity = df.groupby('energy_bins', observed=False)['popularity'].mean()



plt.figure(figsize=(12, 6))

sns.lineplot(x=energy_popularity.index.astype(str), y=energy_popularity.values, marker="o", color="purple")

plt.title("Средняя популярность по интервалам уровня энергичности")

plt.xlabel("Интервалы энергичности")

plt.ylabel("Средняя популярность")

plt.xticks(rotation=45)

plt.show()

**Вывод:**

* Большинство треков находится в диапазоне от 0.6 до 1 по энергичности

* Средняя популярность растет с увеличением энергичности до 0.45 – 0.55, где достигает пика.
  
* После энергичности 0.65 – 0.7 средняя популярность начинает плавно снижаться.

Можно выделить категории:

* Сбалансированные треки (0.4 – 0.8): Наиболее популярные.

* Унылые (0.0 – 0.3): Могут быть использованы в узких жанрах или для специфических аудиторий.

* Ограниченно популярные (0.8 – 1.0): Подходят для танцевальных вечеринок, но ограничены в массовом успехе.

### Зависимость популярности от тональности

In [ ]:
df["key"].describe()

In [ ]:
plt.figure(figsize=(10, 6))

sns.boxplot(data=df, x="key", y="popularity")

plt.title("Зависимость популярности от тональности")

plt.xlabel("Тональность")

plt.ylabel("Популярность")

plt.show()

**Вывод:**

Тональность особо никак не влияет на популярность.

### Зависимость популярности от громкости

In [ ]:
df["loudness"].describe()

In [ ]:
plt.figure(figsize=(10, 6))

sns.kdeplot(data=df, x="loudness", y="popularity", cmap="viridis", fill=True)

plt.title("Плотность зависимости популярности от громкости трека")

plt.xlabel("Громкость")

plt.ylabel("Популярность")

plt.show()

**Вывод:**

Слушатели не любят слишком тихие треки, поэтому популярными становятся средние по громкости.

### Зависимость популярности от модальности

In [ ]:
df["mode"].describe()

In [ ]:
plt.figure(figsize=(8, 6))

sns.barplot(data=df, x="mode", y="popularity")

plt.title("Зависимость популярности от модальности")

plt.xlabel("Модальность")

plt.ylabel("Средняя популярность")

plt.xticks([0, 1], ["Минор", "Мажор"])

plt.show()

**Вывод:**

Модальности никак не влияет на популярность.

### Зависимость популярности от процента слов в песне

In [ ]:
df["speechiness"].describe()

In [ ]:
plt.figure(figsize=(10, 6))

sns.kdeplot(data=df, x="speechiness", y="popularity", cmap="viridis", fill=True)

plt.title("Плотность зависимости популярности от процента слов в песне")

plt.xlabel("Количество слов")

plt.ylabel("Популярность")

plt.show()

**Вывод:**

В основном в треках не так много слов и это особо не влияет на популярность.

### Зависимость популярности от акустичности

In [ ]:
df["acousticness"].describe()

In [ ]:
plt.figure(figsize=(10, 6))

sns.kdeplot(data=df, x="acousticness", y="popularity", cmap="viridis", fill=True)

plt.title("Плотность зависимости популярности от акустичности трека")

plt.xlabel("Акустичность")

plt.ylabel("Популярность")

plt.show()

**Вывод:**

Треки в основном не особо акустичны и это несильно влияет на популярность.

### Зависимость популярности от инструментальности

In [ ]:
df["instrumentalness"].describe()

In [ ]:
plt.figure(figsize=(10, 6))

sns.kdeplot(data=df, x="instrumentalness", y="popularity", cmap="viridis", fill=True)

plt.title("Плотность зависимости популярности от инструментальности трека")

plt.xlabel("Инструментальность")

plt.ylabel("Популярность")

plt.show()

**Вывод:**

Треки по инструментальности можно разделить на две группы (небольшая и большая инструментальность) треки с небольшой популярнее.

### Зависимость популярности от уровня присутствия живой аудитории

In [ ]:
df["liveness"].describe()

In [ ]:
plt.figure(figsize=(10, 6))

sns.kdeplot(data=df, x="liveness", y="popularity", cmap="viridis", fill=True)

plt.title("Плотность зависимости популярности от уровня присутствия живой аудитории")

plt.xlabel("Живая аудитория")

plt.ylabel("Популярность")

plt.show()

**Вывод:**

Особо не влияет.

### Зависимость популярности от валентности

In [ ]:
df["valence"].describe()

In [ ]:
plt.figure(figsize=(10, 6))

sns.kdeplot(data=df, x="valence", y="popularity", cmap="viridis", fill=True)

plt.title("Плотность зависимости популярности от валентности трека")

plt.xlabel("Валентность")

plt.ylabel("Популярность")

plt.show()

**Вывод:**

Треки по валентности распределены примерно равноверно, самая высокая популярность у треков со средим значением.

### Зависимость популярности от темпа

In [ ]:
df["tempo"].describe()

In [ ]:
plt.figure(figsize=(10, 6))

sns.kdeplot(data=df, x="tempo", y="popularity", cmap="viridis", fill=True)

plt.title("Плотность зависимости популярности от темпа трека")

plt.xlabel("Темп")

plt.ylabel("Популярность")

plt.show()

**Вывод:**

Средний темп положительно влияет на популярность.

### Зависимость популярности от размерности

In [ ]:
df["time_signature"].describe()

In [ ]:
plt.figure(figsize=(10, 6))

sns.boxplot(data=df, x="time_signature", y="popularity")

plt.title("Зависимость популярности от размерности")

plt.xlabel("Размерность")

plt.ylabel("Популярность")

plt.show()

**Вывод:**

Большинство треков имеют размерность 4 и это практически никак не влияет на популярность.

### Зависимость популярности от жанра

In [ ]:
df["track_genre"].describe()

In [ ]:
plt.figure(figsize=(25, 6))
avg_popularity_by_genre = df.groupby('track_genre')['popularity'].mean().sort_values(ascending=False)
sns.barplot(x=avg_popularity_by_genre.index, y=avg_popularity_by_genre.values, palette='viridis')
plt.title("Средняя популярность песен по жанрам")
plt.xticks(rotation=90)
plt.xlabel("Жанр")
plt.ylabel("Средняя популярность")
plt.show()

**Вывод:** 

Самые популярные жанры на тот момент - pop film, k-pop, chill и sad. Жанр явно влияет на популярность песни.

### Кореляция числовых данных

In [ ]:
plt.figure(figsize=(12, 8))

correlation_matrix = df[["danceability", "energy", "loudness", "speechiness", "acousticness",

                         "instrumentalness", "liveness", "valence", "tempo"]].corr()

sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1)

plt.title("Карта корреляций между музыкальными характеристиками")

plt.show()

**Вывод:**

* Наблюдается высокая положительная корреляция между energy и loudness, что логично, так как более громкие треки обычно передают больше энергии.
  
* acousticness имеет отрицательную корреляцию с energy и loudness, что показывает, что акустические треки обычно тише и менее энергичны.

* Низкая или нулевая корреляция между speechiness, liveness и другими характеристиками указывает на то, что эти признаки имеют независимые вариации и могут описывать уникальные аспекты трека.

## Feature engineering

### 1. Категории продолжителности

In [ ]:
df['duration_category'] = pd.cut(df['duration_s'], bins=[0, 120, 240, 2000], labels=['short', 'standart', 'long'])

* short - [0; 120] секунд
* standart - (120; 240] секунд
* long - (240; ...] секунд

Категории могут помочь лучше понять различия в популярности треков с различной продолжительностью.

### 2. Категории танцевальности

In [ ]:
df['danceability_category'] = pd.cut(df['danceability'], bins=[0, 0.4, 0.7, 1], labels=['low', 'balanced', 'high'])


* Сложные для восприятия [0.0; 0.4]: Могут быть использованы в узких жанрах или для специфических аудиторий.
* Сбалансированные треки (0.4; 0.7]: Наиболее популярные.
* Ограниченно популярные (0.7; 1.0]: Подходят для танцевальных вечеринок, но ограничены в массовом успехе.
ехе.

### 2. Категории энергичности

In [ ]:
df['energy_category'] = pd.cut(df['energy'], bins=[0, 0.4, 0.8, 1], labels=['calm', 'balanced', 'active'])


* Спокойные [0.0; 0.4]: Могут быть использованы в узких жанрах или для специфических аудиторий.
* Сбалансированные треки (0.4; 0.8]: Наиболее популярные.
* Активные (0.8; 1.0]: Подходят для танцевальных вечеринок, но ограничены в массовом успехе
пехе

### 3. Нормализация громкости

In [ ]:
df['loudness_normalized'] = (df['loudness'] - df['loudness'].min()) / (df['loudness'].max() - df['loudness'].min())

Приводит громкость к диапазону от 0 до 1. Нормализация важна, так как она устраняет эффект масштаба и делает признак пригодным для моделей, чувствительных к масштабированию.

### 3. Отношения вокальности к инструментальности

In [ ]:
df['speech_instrument_ratio'] = df['speechiness'] / (df['instrumentalness'] + 1e-9)

speech_instrument_ratio — отношение вокального компонента к инструментальному. Высокое значение показывает, что трек больше сосредоточен на вокале, чем на инструментальных фрагментах.

### 4. Отношения акустичности к инструментальности

In [ ]:
df['acoustic_instrument_ratio'] = df['acousticness'] / (df['instrumentalness'] + 1e-9)

acoustic_instrument_ratio — соотношение акустического компонента и инструментальности. Высокие значения показывают, что трек больше зависит от акустики, чем от инструментальности.

### 5. Отношения танцевальности к энергичности

In [ ]:
df['dance_energy_ratio'] = df['danceability'] / (df['energy'] + 1e-9)

dance_energy_ratio — отношение танцевальности к энергии, чтобы выяснить, насколько сильно энергетика трека влияет на его танцевальность.

### 6. Позитивность

In [ ]:
df['positivity'] = df['energy'] * df['valence']

Позитивность - это произведение показателей энергии и валентности (настроения), которое может характеризовать общую "весёлость" или позитивную энергию трека.

## Исправление типов

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
numerical_features = df.select_dtypes(include=['number', 'float64', 'int64']).columns.tolist()

categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()

In [ ]:
categorical_features

In [ ]:
from sklearn.preprocessing import LabelEncoder

for col in categorical_features:

    le = LabelEncoder()

    df[col] = le.fit_transform(df[col])

## Эксперименты с моделями

### 1. Определение метрик


* MAE (Mean Absolute Error) — средняя абсолютная ошибка. Показывает среднее отклонение предсказаний от фактических значений.
* MSE (Mean Squared Error) — среднеквадратическая ошибка. Возводит отклонения в квадрат, делая крупные ошибки более заметными.
* RMSE (Root Mean Squared Error) — корень из среднеквадратической ошибки. Корень из MSE, возвращая ошибку к изначальной размерности.
* MAPE (Mean Absolute Percentage Error) — средняя абсолютная процентная ошибка. Выражает среднюю абсолютную ошибку в процентах, что удобно для интерпретации. (Убрал т.к. показывает плохие значения из-за нулевых значений в таргете)
* R² (R-Squared) — коэффициент детерминации. Доля дисперсии, объяснённой моделью (чем ближе к 1, тем лучше).чше).

### 2. Разделение данных на обучающую и тестовую выборки

In [ ]:
# Разделение данных на обучающую и тестовую выборки

X = df.drop(columns=['popularity'])

y = df['popularity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error


def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    r2 = r2_score(y_true, y_pred)
    
    metrics = {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2': r2
    }
    return metrics

### 3. Нормализация данных

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

### Эксперимент 1: Линейная модель

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_predict

# Модель
linear_model = LinearRegression()

# Кросс-валидация
y_pred = cross_val_predict(linear_model, X_train, y_train, cv=5)

# Оценка
metrics_linear = evaluate_model(y_train, y_pred)
print('Линейная модель:', metrics_linear)

Плюсы:
- MAE: Средняя абсолютная ошибка 18.28 указывает на допустимое среднее отклонение предсказаний от истинных значений.
- R²: Коэффициент детерминации (0.031) положительный, но очень близок к нулю, что говорит о том, что модель объясняет очень малую часть дисперсии целевой переменной.

Минусы:
- RMSE: Высокое значение (21.96) говорит о крупных ошибках.
- MAPE: Значение равно inf, что вызвано наличием нулевых значений в целевой переменной (y). 

Вывод:
Линейная модель плохо справляется с задачей. Низкое значение R² и высокий RMSE свидетельствуют о необходимости улучшений.

### Эксперимент 2: Дерево решений

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Модель
tree_model = DecisionTreeRegressor(random_state=42)

# Кросс-валидация
y_pred = cross_val_predict(tree_model, X_train, y_train, cv=5)

# Оценка
metrics_tree = evaluate_model(y_train, y_pred)
print('Дерево решений:', metrics_tree)

Плюсы:

* MAE: Снижение средней абсолютной ошибки до 13.79 (по сравнению с линейной моделью — 18.28) говорит о лучшем качестве предсказаний.
* MSE и RMSE: Незначительное улучшение (RMSE: 21.72 против 21.96 у линейной модели), но этого недостаточно для значимого прогресса.
* R²: Коэффициент детерминации (0.052) немного выше, чем у линейной модели (0.031), но по-прежнему очень низкий, что указывает на слабую объяснительную способность.



Минусы:
* MAPE: Остался inf из-за нулей в целевой переменной.

Вывод:

Дерево решений лучше справляется с задачей, чем линейная модель, особенно по метрике MAE.
Однако, общее качество модели по-прежнему оставляет желать лучшего, о чем свидетельствуют высокий RMSE и низкий R².
Учитывая природу задачи (смешение нулей и ненулей в целевой переменной), деревья решений могут быть недостаточно гибкими.

### Эксперимент 3: Градиентный бустинг

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Модель
boosting_model = GradientBoostingRegressor(random_state=42)

# Кросс-валидация
y_pred = cross_val_predict(boosting_model, X_train, y_train, cv=5)

# Оценка
metrics_boosting = evaluate_model(y_train, y_pred)
print('Градиентный бустинг:', metrics_boosting)

**Плюсы:**

* MAE: Средняя абсолютная ошибка (15.98) показывает снижение ошибки предсказаний относительно линейной модели (18.28).
* R²: Коэффициент детерминации (0.215) заметно выше, чем у линейной модели (0.031) и дерева решений (0.052), что говорит о лучшей объяснительной способности модели.
* RMSE: Снижение ошибки до 19.76 по сравнению с линейной моделью (21.96) и деревом решений (21.72) указывает на меньшее количество крупных ошибок.Минусы:


**Вывод:**

Градиентный бустинг показывает значительное улучшение по сравнению с линейной моделью и деревом решений, особенно по метрике R². 

### Эксперимент 4: xgboost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Создание модели XGBoost
xgboost_model = xgb.XGBRegressor(n_estimators=50, random_state=42)

# Кросс-валидация с 3 фолдами
y_pred = cross_val_predict(xgboost_model, X_train, y_train, cv=5)

# Оценка модели
metrics_xgboost = evaluate_model(y_train, y_pred)
print('XGB бустинг:',metrics_xgboost)

**Плюсы:**


* MAE: Самая низкая ошибка среди всех моделей (12.90), что свидетельствует о высокой точности предсказаний.
* R²: Коэффициент детерминации (0.42) значительно превышает результаты остальных моделей, что говорит о лучшей объяснительной способности.
* RMSE: Минимальная ошибка (16.99) показывает наименьшие отклонения в крупных ошибках по сравнению с другими моделями.



**Вывод:**

XGB бустинг демонстрирует лучший результат по всем метрикам среди рассмотренных моделей. Низкие значения MAE и RMSE, а также высокий R² указывают на значительное улучшение качества предсказаний. 

### Эксперимент 5: Нейронная сеть

#### 1. Создание модели

In [ ]:
from tensorflow.keras import regularizers

nn_model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],),
                      kernel_regularizer=regularizers.l2(0.2)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.2)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1)
])

#### 2. Компиляция модели

In [ ]:
nn_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

#### 4. Тренировка модели

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-6)
history = nn_model.fit(
    X_train, 
    y_train, 
    epochs=100, 
    batch_size=32, 
    verbose=1,
    validation_split=0.2,
    validation_data=(X_train, y_train), 
    callbacks=[early_stopping, reduce_lr]
)

#### 5. Оценивание модели

In [ ]:
y_pred = nn_model.predict(X_train).flatten()

metrics_nn = evaluate_model(y_train, y_pred)
print('Нейронная сеть:', metrics_nn)

# Построение графика лосса
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Training Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
plt.title('Loss During Training and Validation with Regularization and Dropout')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

**Плюсы:**


* MAE: Средняя абсолютная ошибка (17.86) близка к результату линейной модели (18.28), что указывает на допустимый уровень точности.
* R²: Коэффициент детерминации (0.064) немного выше, чем у линейной модели (0.031) и дерева решений (0.052).


**Минусы:**

* RMSE: Высокая ошибка (21.58) почти на уровне линейной модели (21.96) и дерева решений (21.72), что указывает на значительные отклонения.


**Вывод:**

Нейронная сеть не показала значительных улучшений по сравнению с линейной моделью и деревом решений. Несмотря на небольшое повышение R², высокое значение RMSE говорит о необходимости доработки модели для повышения её предсказательной способности.

### Оценка ввжности признаков для xgboost_model

In [ ]:
# Важность признаков с помощью permutation importance
xgboost_model.fit(X_train, y_train)
result = permutation_importance(xgboost_model, X_test, y_test, n_repeats=10, random_state=42, scoring='neg_mean_absolute_error')

importances = pd.Series(result.importances_mean, index=X.columns)



# Визуализация важности признаков

importances.sort_values().plot(kind='barh', figsize=(10,6))

plt.title('Важность признаков')

plt.xlabel('Среднее уменьшение MAE')

plt.show()

**Ключевые признаки:**


* track_genre (жанр трека) имеет наибольшее влияние на предсказание модели. Это может указывать на то, что популярность песни сильно зависит от жанра.
* album_name (название альбома) и artists (исполнители) также играют значительную роль, что говорит о влиянии узнаваемости альбома или исполнителя на популярность.


**Акустические характеристики:**

* Такие признаки, как acousticness (акустичность), loudness (громкость) и instrumentalness (инструментальность), заметно влияют на результаты модели. Это подтверждает, что технические характеристики песни важны для её популярности.


**Второстепенные признаки:**


* Признаки вроде danceability (танцевальность), energy (энергия) и valence (эмоциональный тон) также играют некоторую роль, но их влияние значительно меньше.
* Характеристики, связанные с длительностью песни и тональностью (mode, key), имеют минимальное влияние, что может означать их слабую корреляцию с популярностью.


## Заключание

Результаты анализа показывают, что популярность трека зависит как от характеристик, связанных с жанром и громкостью, так и от эмоциональных и вокальных характеристик. Эти результаты можно использовать для:


* Оптимизации треков перед релизом.
* Формирования рекомендаций для исполнителей и продюсеров.
* Разработки рекомендационных систем на основе данных о предпочтениях пользователей.


Но не смотря на всё это мы не можем точно предсказывать будет трек популярным или нет, потому что популярность - метрика практически непредсказуемая, она зависит от слишком большого количества факторов, которые невозможно в полной мере собрать для каждого трека.
